encoding utf-8 

@File       : .py
@Contact    : username.you@email.address.com
@License    : (C)Copyright AIQ Solutions 2022

@ModifyTime     @Author     @Version        @Description
-----------     -------     --------        ------------
  username    1.0             None

Module Documentation: 

In [16]:
import pydicom
from dcmrtstruct2nii import dcmrtstruct2nii, list_rt_structs
import dicom2nifti
import nibabel as nib
import nrrd
import glob
import os
import numpy as np
from collections import OrderedDict
import shutil


In [2]:
# import warnings
# warnings.filterwarnings("ignore")

In [17]:
root_study_dir = 'D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake'
# output_root = 'D:\\Prostate02_Test_Retest\\nii_test_retest'

In [6]:
# # remove space in folder name
# scan_dir = os.listdir(root_study_dir)
# scan_dir.sort()
# i = 0
# for scan in scan_dir:
#     os.rename(os.path.join(root_study_dir,scan),os.path.join(root_study_dir,scan.replace(" ", "")))

In [18]:
# remove nii and nrrd folders
scan_dir = os.listdir(root_study_dir)
scan_dir.sort()
# i = 0
for scan in scan_dir:
    if "." not in scan:
        print("Now processing:" + str(scan))

        scan_modal = os.listdir(os.path.join(root_study_dir,scan))
        scan_modal.sort()
        for scan_modal_each in scan_modal:
            if scan_modal_each[0] != "." and os.path.isdir(os.path.join(root_study_dir,scan,scan_modal_each)):
                if scan_modal_each == "nii" or scan_modal_each == "nrrd":
                    # print(scan_modal_each)
                    shutil.rmtree(os.path.join(root_study_dir,scan,scan_modal_each))
 

Now processing:RP-01 1
Now processing:RP-01 2
Now processing:RP-02 1
Now processing:RP-02 2
Now processing:RP-03 1
Now processing:RP-03 2
Now processing:RP-04 1
Now processing:RP-04 2
Now processing:RP-05 1
Now processing:RP-05 2
Now processing:RP-06 1
Now processing:RP-06 2
Now processing:RP-08 1
Now processing:RP-08 2
Now processing:RP-09 1
Now processing:RP-09 2
Now processing:RP-10 1
Now processing:RP-10 2
Now processing:RP-13 1
Now processing:RP-13 2
Now processing:RP-14 1
Now processing:RP-14 2
Now processing:RP-15 1
Now processing:RP-15 2
Now processing:RP-17 1
Now processing:RP-17 2
Now processing:RP-20 1
Now processing:RP-20 2
Now processing:RP-21 1
Now processing:RP-21 2
Now processing:RP-23 1
Now processing:RP-23 2
Now processing:RP-24 1
Now processing:RP-24 2
Now processing:RP-25 1
Now processing:RP-25 2


In [19]:
scan_dir = os.listdir(root_study_dir)
scan_dir.sort()
# i = 0
for scan in scan_dir:
    if "." not in scan:
        print("Now processing:" + str(scan))

        scan_modal = os.listdir(os.path.join(root_study_dir,scan))
        scan_modal.sort()
        for scan_modal_each in scan_modal:
            if scan_modal_each[0] != "." and os.path.isdir(os.path.join(root_study_dir,scan,scan_modal_each)):
                # print(scan_modal_each)


                if '_RTst_' in scan_modal_each:
                    os.mkdir(os.path.join(root_study_dir,scan,"nii"))
                    output_folder = os.path.join(root_study_dir,scan,"nii",scan_modal_each)
                    os.mkdir(output_folder)
                    full_path = os.path.join(root_study_dir,scan,scan_modal_each)
                    rt_file = sorted(glob.glob(full_path + '\\*.dcm'))
                    dicom_file_path = sorted(glob.glob(os.path.join(root_study_dir,scan) + '\\*_CT_*'))
                    print("Using: " + str(dicom_file_path))
                    # converts rt struct dicom file to nii but needs dicom folder
                    dcmrtstruct2nii(rt_file[0], dicom_file_path[0], output_folder)
                    print('written RT: ', output_folder)

Now processing:RP-01 1
Using: ['D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake\\RP-01 1\\2.16.840_681.3491_CT_2020-07-22_112045_PET.PSMA_CT.Soft.Tissue.Whole.Body_n221__00000']
written RT:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nii\RP-01 1_RP-01_RTst_2020-07-22_112045_PET.PSMA_Saved.Session.2TP.complete_n1__00000
Now processing:RP-01 2
Using: ['D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake\\RP-01 2\\2.16.840_079.5471_CT_2020-07-24_152428_PET.PSMA_CT.Soft.Tissue.Whole.Body_n221__00000']
written RT:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 2\nii\RP-01 2_RP-01_RTst_2020-07-24_152428_PET.PSMA_Saved.Session.2TP.complete_n1__00000
Now processing:RP-02 1
Using: ['D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake\\RP-02 1\\2.16.840_649.4692_CT_2020-07-22_114957_PET.PSMA_CT.Soft.Tissue.Whole.Body_n221__00000']
written RT:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Ret

In [20]:
def generate_nrrd_header(ds,img3d):
    # type, dimensions, space = 'left-posterior-superior', sizes, space direction, encoding, endian, kinds, space origin
    z_max_pos = ds[0].ImagePositionPatient[2]
    # z_list = []
    for ds_ct in ds:
        # z_list.append(ds_ct.ImagePositionPatient[2])
        if ds_ct.ImagePositionPatient[2] < z_max_pos:
            z_max_pos = ds_ct.ImagePositionPatient[2]

    # print(min(z_list))
    image_start = np.asarray((ds[0].ImagePositionPatient[0], ds[0].ImagePositionPatient[1], z_max_pos))
    nrrd_header = OrderedDict()

    nrrd_header['type'] = 'short'  # this gets overridden by the img3d data type
    nrrd_header['dimension'] = 3
    nrrd_header['space'] = 'left-posterior-superior'

    nrrd_header['sizes'] = np.array(img3d.shape)
    nrrd_header['space directions'] = np.array([[ds[0].PixelSpacing[0], 0, 0],
                                                [0, ds[0].PixelSpacing[1], 0],
                                                [0, 0, ds[0].SliceThickness]])
    nrrd_header['encoding'] = 'gzip'
    nrrd_header['endian'] = 'little'
    nrrd_header['kinds'] = ('domain', 'domain', 'domain')
    nrrd_header['space origin'] = np.array(image_start)

    return nrrd_header

In [5]:
# root_study_dir = 'D:\\Melanoma01\\FinalTrainingCohort_AIQ_Mel01_Corrected\\Final_Mel01_nii'
# output_root = 'D:\\Melanoma01\\FinalTrainingCohort_AIQ_Mel01_Corrected\\Final_Mel01_nrrd'

In [23]:
subjects_scans = [os.path.basename(f) for f in sorted(glob.glob(root_study_dir + '\\RP*'))]
for sub in subjects_scans:
    print('subject: ', sub)
    nii_files = sorted(glob.glob(root_study_dir + "\\"+ sub + "\\" + "nii" + "\\*RTst*" '\\*.nii.*'))

    dicom_dir = sorted(glob.glob(os.path.join(root_study_dir, sub) + '\\*_CT_*'))
    dicom_dir = list(filter(lambda dir: dir[0] != '.' and "nii" not in dir and "PT" not in dir, dicom_dir))

    dicom_file = [pydicom.dcmread(f) for f in sorted(glob.glob(dicom_dir[0] + '\\2*.dcm'))]
    output_folder = os.path.join(root_study_dir, sub,"nrrd")
    os.mkdir(output_folder)
    # print('nii files: ', nii_files)
    print('ref dicom: ', dicom_dir)

    for nii_file in nii_files:
        a = nib.load(nii_file)
        image3d = a.get_fdata()
        header_info = generate_nrrd_header(dicom_file, image3d)
        output_file_name = os.path.join(output_folder, os.path.basename(nii_file).split('.')[0] + '.nrrd')
        nrrd.write(output_file_name, image3d, header_info)
        print('wrote: ', output_file_name)

subject:  RP-24 2
ref dicom:  ['D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake\\RP-24 2\\1.3.51.0_12904926_CT_2021-07-16_143834_PET.NC.F18.PSMA_CT.Soft.Tissue.Whole.Body_n246__00000']
wrote:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-24 2\nrrd\image.nrrd
wrote:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-24 2\nrrd\mask_19-1.nrrd
wrote:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-24 2\nrrd\mask_Ref_Blood-Pool.nrrd
wrote:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-24 2\nrrd\mask_Ref_Gluteus-Maximus.nrrd
wrote:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-24 2\nrrd\mask_Ref_LPI.nrrd
wrote:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-24 2\nrrd\mask_Ref_Liver.nrrd
wrote:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-24 2\nrrd\mask_Ref_LspineL1-4.nrrd
wrote:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Tes

In [11]:
source_folder = "D:\\Prostate02_Test_Retest\\Prostate02_Nrrd PET+CT\\Nrrd PET+CT"
destination = "D:\\Prostate02_Test_Retest\\test_retest02\\PatientData_Processed"

In [15]:
dirs = os.listdir(source_folder)
dirs.sort()
i = 1
for subjects in dirs:
    if not os.path.exists(os.path.join(destination,subjects)):
        os.mkdir(os.path.join(destination,subjects))
    sessions = os.listdir(os.path.join(source_folder,subjects))
    sessions.sort()
    for session in sessions:
        if i % 2 == 1:
            scan_folder = subjects.replace('-','_') + "_scan01"
            if not os.path.exists(os.path.join(destination,subjects,scan_folder)):
                os.mkdir(os.path.join(destination,subjects,scan_folder))
        else:
            scan_folder = subjects.replace('-','_') + "_scan02"
            if not os.path.exists(os.path.join(destination,subjects,scan_folder)):
                os.mkdir(os.path.join(destination,subjects,scan_folder))
        for file in os.listdir(os.path.join(source_folder,subjects,session)):
            if ".json" not in file and "ct" in file:
                shutil.copyfile(os.path.join(source_folder,subjects,session,file), os.path.join(destination,subjects,scan_folder,scan_folder + "_ctData.nrrd"))
            if ".json" not in file and "pt" in file:
                shutil.copyfile(os.path.join(source_folder,subjects,session,file), os.path.join(destination,subjects,scan_folder,scan_folder + "_petData.nrrd"))
        i += 1

In [24]:
source_folder = "D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake"
destination = "D:\\Prostate02_Test_Retest\\test_retest02\\PatientData_Processed"

In [25]:
#remove nrrd (name error)
for patient in os.listdir(destination):
    for scans in os.listdir(os.path.join(destination,patient)):
        for files in os.listdir(os.path.join(destination,patient,scans)):
            if "lesion" in files:
                os.remove(os.path.join(destination, patient,scans,files))

In [20]:
# for subjects in os.listdir(destination):
#     os.rename(os.path.join(destination,subjects), os.path.join(destination,subjects.replace('-', '_')))

In [26]:
dirs = os.listdir(source_folder)
dirs.sort()
for subjects in dirs:
    if '.' not in subjects:
        print("Now processing:" + str(subjects))
        scans = os.listdir(os.path.join(source_folder, subjects))
        scans.sort()
        for scan in scans:
            if 'nrrd' in scan:
                for files in os.listdir(os.path.join(source_folder, subjects,scan)):
                    if 'lesionData' in files:
                        print(os.path.join(source_folder, subjects,scan,files))
                        subject = subjects.split(' ')[0].replace('-', '_')
                        scan_no = subjects.split(' ')[1]
                        scan_folder = subject + '_scan0' + str(scan_no)
                        shutil.copyfile(os.path.join(source_folder,subjects,scan,files), os.path.join(destination,subject,scan_folder,scan_folder + "_lesionData.nrrd"))


Now processing:RP-01 1
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\RP_01_lesionData.nrrd
Now processing:RP-01 2
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 2\nrrd\RP_01_lesionData.nrrd
Now processing:RP-02 1
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-02 1\nrrd\RP_02_lesionData.nrrd
Now processing:RP-02 2
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-02 2\nrrd\RP_02_lesionData.nrrd
Now processing:RP-03 1
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-03 1\nrrd\RP_03_lesionData.nrrd
Now processing:RP-03 2
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-03 2\nrrd\RP_03_lesionData.nrrd
Now processing:RP-04 1
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-04 1\nrrd\RP_04_lesionData.nrrd
Now processing:RP-04 2
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-04 2\nrrd\RP_04_lesionData.nrrd
Now processing:RP-05 1
D